In [2]:
import pandas as pd 
import pyodbc
from datetime import datetime 

In [3]:
#função de conexão entre python e SQL

def retorna_conexao_sql():
    Server= "DESKTOP-33I2UG6\SQLEXPRESS;"
    Database="mini_Projeto_heloo;"
    string_conexao = 'Driver={SQL Server Native Client 11.0};Server='+Server+';Database='+Database+';Trusted_Connection=yes;'
    conexao = pyodbc.connect(string_conexao)
    return conexao

In [4]:
con = retorna_conexao_sql()

In [5]:
#traformando a query paciente em um dataFrame

dim_paciente = 'SELECT * FROM pacientes;'
df_paciente = pd.read_sql(dim_paciente, con)
df_paciente.head()

,id,nome,telefone
0,1,Geralda Garcia de Lima,(83)99123-4589
1,2,Fernado da Silva Oliveira,(83)99947-3574
2,3,Maria dos Santos Pereira,(83)98623-1688
3,4,Julia Cristina Bastista,(83)99677-4241
4,5,Pedro Vinicius Fenandes,(83)99658-3340


In [6]:
#transformando as primeiras letras do nome em maiuscula dim_Paciente
df_paciente['nome'] = df_paciente['nome'].apply(lambda x: x.title())

In [7]:
#Calculando o total de Paciente 

total_Paciente = df_paciente['id'].count()
total_Paciente

55

In [8]:
#traformando a query medicos em um dataFrame

dim_medico = 'SELECT * FROM medicos;'
df_medico = pd.read_sql(dim_medico, con)
df_medico

,id,crm,nome,especialidade,telefone
0,1,CRM1010/PB,Ariana Souza de Lima,Ginecologista,(83)99981-2383
1,2,CRM1345/PB,José Ivo dos Santos,Reumatologista,(83)99169-1320
2,3,CRM133/PB,Matheus OLiveira da Silva,Neurologista,(83)98699-3639
3,4,CRM1456/PB,Junior guedes Maciel,Cardiologista,(83)98862-0943
4,5,CRM1237/PB,Vinicius Lisboa,Dermatologista,(83)99926-4737
5,6,CRM1906/PB,Caetaneo Pereira,Clinico Geral,(83)98876-5617


In [9]:
#Calculando o total de medicos 
total_medico = df_medico['id'].count()
total_medico

6

In [10]:
#tranformando a query fato em dataFrame
fato = 'SELECT * FROM consultas;'
df_fato = pd.read_sql(fato , con)
df_fato.head()

,id,paciente_id,medico_id,data_agendamento,situacao_id,cidade_id
0,1,1,1,2021-02-18,1,1
1,2,2,2,2021-02-18,2,2
2,3,3,4,2021-02-18,3,3
3,4,4,1,2021-02-18,2,4
4,5,5,4,2021-02-18,3,5


In [11]:
df_fato.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                55 non-null     int64 
 1   paciente_id       55 non-null     int64 
 2   medico_id         55 non-null     int64 
 3   data_agendamento  55 non-null     object
 4   situacao_id       55 non-null     int64 
 5   cidade_id         55 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.7+ KB


In [12]:
#transformando String para dateTime 
df_fato['data_agendamento'] =  pd.to_datetime (df_fato['data_agendamento'], infer_datetime_format = True ) 

In [13]:
consultas_canceladas = df_fato[df_fato['situacao_id'] == 3]['id']
consultas_canceladas.count()

24

In [14]:
consultas_realizadas_por_mes= df_fato[df_fato['situacao_id'] == 2]['data_agendamento'].dt.month
consultas_realizadas_por_mes.value_counts()

4    13
3     8
2     2
Name: data_agendamento, dtype: int64

In [15]:
medico_mais_cancelam = df_fato[df_fato['situacao_id'] == 3]['medico_id']
medico_mais_cancelam.value_counts()

4    7
6    6
5    5
3    3
1    2
2    1
Name: medico_id, dtype: int64

In [16]:
consultas_canc_por_mes = df_fato[df_fato['situacao_id'] == 3].groupby(['data_agendamento','medico_id'])
consultas_canc_por_mes.count()

id  paciente_id  situacao_id  cidade_id
data_agendamento medico_id                                         
2021-02-18       4           2            2            2          2
                 5           3            3            3          3
                 6           1            1            1          1
2021-03-18       1           1            1            1          1
                 3           1            1            1          1
                 4           1            1            1          1
2021-04-18       1           1            1            1          1
                 2           1            1            1          1
                 3           2            2            2          2
                 4           4            4            4          4
                 5           2            2            2          2
                 6           5            5            5          5

In [18]:
cidades_mais_canceladas = df_fato[df_fato['situacao_id'] == 3]['cidade_id']
cidades_mais_canceladas.value_counts(ascending=False)

1    9
3    7
5    4
2    2
7    1
6    1
Name: cidade_id, dtype: int64

In [20]:
#transformando a query dim_situacao em dataFrame
dim_situacao = 'SELECT * FROM consultas_situacoes;'
df_situacao= pd.read_sql(dim_situacao , con)
df_situacao

,id,nome
0,1,agendada
1,2,executada
2,3,cancelada


In [21]:
#Transformando a primeira letra do nome em maiusculo
df_situacao['nome'] = df_situacao['nome'].apply(lambda x: x.capitalize())
df_situacao

,id,nome
0,1,Agendada
1,2,Executada
2,3,Cancelada


In [22]:
#tranformando a query dim_Cidade em dataFrame
dim_cidade = 'SELECT * FROM cidades;'
df_cidade= pd.read_sql(dim_cidade , con)
df_cidade

,id,nome,estado_sigla
0,1,João Pessoa,PB
1,2,Guarabira,PB
2,3,Campina Grande,PB
3,4,Mamanguape,PB
4,5,Araruna,PB
5,6,Rio Tinto,PB
6,7,Alagoa Grande,PB


In [23]:
import plotly.express as px

In [58]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [34]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


In [35]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [42]:
df1 = pd.DataFrame(cidades_mais_canceladas)
df2 = pd.DataFrame(consultas_realizadas_por_mes)
df1.head()

,cidade_id
2,3
4,5
7,1
11,5
12,5


In [51]:

app.title = 'Analytics Dashboard'
app.layout = html.Div()


In [61]:
if __name__ == '__main__':     
   app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [74]:
fig = px.bar(df_fato, x='cidade_id', y='data_agendamento')
html.Div(
	[
		html.H6(
			"Aqui temos um exemplo de gráfico Bar Charts", className="subtitle padded"
		),
		dcc.Graph(
			
			figure=fig
		)
	],
	className="img",

)
if __name__ == '__main__':     
   app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
